In [3]:
import collections
import os
import random
import time
from tqdm import tqdm
import torch
from torch import nn
import pandas as pd
import numpy as np
import torchtext.vocab as Vocab
import torch.utils.data as Data
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
train_data = pd.read_csv('data/train_data.csv')
test_data = pd.read_csv('data/test_data.csv')

In [5]:
def get_tokenized_imdb(data):
    '''
    @params:
        data: 数据pd
    @return: 切分词后的文本的列表，列表中的每个元素为切分后的词序列
    '''
    def tokenizer(text):
        return [tok.lower() for tok in str(text).split(' ')]
    
    return [tokenizer(review) for review in list(data['text'])]

def get_vocab_imdb(data):
    '''
    @params:
        data: 同上
    @return: 数据集上的词典，Vocab 的实例（freqs, stoi, itos）
    '''
    tokenized_data = get_tokenized_imdb(data)
    counter = collections.Counter([tk for st in tokenized_data for tk in st])
    return Vocab.Vocab(counter, min_freq=5)

vocab = get_vocab_imdb(train_data)
print('# words in vocab:', len(vocab))

def preprocess_imdb(data, vocab, is_train):
    '''
    @params:
        data: 同上，原始的读入数据
        vocab: 训练集上生成的词典
    @return:
        features: 单词下标序列，形状为 (n, max_l) 的整数张量
        labels: 情感标签，形状为 (n,) 的0/1整数张量
    '''
    max_l = 30  # 将每条评论通过截断或者补0

    def pad(x):
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))

    tokenized_data = get_tokenized_imdb(data)
    features = np.asarray([pad([vocab.stoi[word] for word in words]) for words in tokenized_data])
    if is_train == 1:
        labels = np.asarray(list(data['target']))
    else:
        labels = 0
    return features, labels

# words in vocab: 2460


In [6]:
text_data,target_data = preprocess_imdb(train_data, vocab, 1)
train_rate = 0.98
train_text_splited = torch.LongTensor(text_data[:int(train_rate*train_data.shape[0]),::])
train_keyword_splited = torch.FloatTensor(np.asarray(pd.get_dummies(train_data['keyword']))[:int(train_rate*train_data.shape[0]),::])
train_target_splited = torch.LongTensor(target_data[:int(train_rate*train_data.shape[0])])
val_text_splited = torch.LongTensor(text_data[int(train_rate*train_data.shape[0]):,::])
val_keyword_splited = torch.FloatTensor(np.asarray(pd.get_dummies(train_data['keyword']))[int(train_rate*train_data.shape[0]):,::])
`b = torch.LongTensor(target_data[int(train_rate*train_data.shape[0]):])

train_set = Data.TensorDataset(train_text_splited,train_keyword_splited,train_target_splited)
val_set = Data.TensorDataset(val_text_splited,val_keyword_splited,val_target_splited)

In [77]:
text_data_test,_ = preprocess_imdb(test_data, vocab, 0)
test_text = torch.LongTensor(text_data_test)
test_keyword = torch.FloatTensor(np.asarray(pd.get_dummies(test_data['keyword'])))


In [78]:
batch_size = 64
train_iter = Data.DataLoader(train_set, batch_size, shuffle=True)
val_iter = Data.DataLoader(val_set, batch_size)

In [92]:
class BiRNN(nn.Module):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers, keywords_num):
        '''
        @params:
            vocab: 在数据集上创建的词典，用于获取词典大小
            embed_size: 嵌入维度大小
            num_hiddens: 隐藏状态维度大小
            num_layers: 隐藏层个数
        '''
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(len(vocab), embed_size)
        
        # encoder-decoder framework
        # bidirectional设为True即得到双向循环神经网络
        self.encoder = nn.LSTM(input_size=embed_size, 
                                hidden_size=num_hiddens, 
                                num_layers=num_layers,
                                bidirectional=True,
                                )
        self.decoder = nn.Linear(4*num_hiddens, 2) # 初始时间步和最终时间步的隐藏状态作为全连接层输入
        
        self.key_fc = nn.Linear(keywords_num, 1)

    def forward(self, inputs, inputs2):
        '''
        @params:
            inputs: 词语下标序列，形状为 (batch_size, seq_len) 的整数张量
        @return:
            outs: 对文本情感的预测，形状为 (batch_size, 2) 的张量
        '''
        # 因为LSTM需要将序列长度(seq_len)作为第一维，所以需要将输入转置
        embeddings = self.embedding(inputs.permute(1, 0)) # (seq_len, batch_size, d)
        #embeddings = F.dropout2d(embeddings)
        # rnn.LSTM 返回输出、隐藏状态和记忆单元，格式如 outputs, (h, c)
        outputs, _ = self.encoder(embeddings) # (seq_len, batch_size, 2*h)
        encoding = torch.cat((outputs[0], outputs[-1]), -1) # (batch_size, 4*h)
        #encoding = F.dropout(encoding)
        #outs = torch.sigmoid(self.decoder(encoding)) # (batch_size, 2)
        
        keywords = torch.sigmoid(self.key_fc(inputs2))
        keywords = encoding.mul(keywords)
        keywords = F.relu(keywords+encoding)
        outs = torch.sigmoid(self.decoder(keywords)) # (batch_size, 2)
        return outs


In [93]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for (X1,X2, y) in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(X1.to(device),X2.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train()
            else:
                if('is_training' in net.__code__.co_varnames):
                    acc_sum += (net(X1,X2, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X1,X2).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n

def train(train_iter, test_iter, net, loss, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for (X1,X2, y) in train_iter:
            X1 = X1.to(device)
            y = y.to(device)
            y_hat = net(X1,X2)
            l = loss(y_hat, y) 
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))
        if test_acc>0.86:
            break

In [94]:
embed_size, num_hiddens, num_layers = 3, 3, 1
net = BiRNN(vocab, embed_size, num_hiddens, num_layers, train_keyword_splited.shape[1])

lr, num_epochs = 0.01, 100
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=lr)
loss = nn.CrossEntropyLoss()

train(train_iter, val_iter, net, loss, optimizer, device, num_epochs)

training on  cpu
epoch 1, loss 0.6696, train acc 0.596, test acc 0.778, time 3.1 sec
epoch 2, loss 0.2798, train acc 0.751, test acc 0.830, time 3.0 sec
epoch 3, loss 0.1675, train acc 0.808, test acc 0.810, time 3.0 sec
epoch 4, loss 0.1201, train acc 0.831, test acc 0.817, time 3.9 sec
epoch 5, loss 0.0928, train acc 0.850, test acc 0.850, time 2.9 sec
epoch 6, loss 0.0753, train acc 0.862, test acc 0.830, time 3.3 sec
epoch 7, loss 0.0633, train acc 0.872, test acc 0.843, time 3.0 sec
epoch 8, loss 0.0548, train acc 0.875, test acc 0.843, time 3.4 sec
epoch 9, loss 0.0480, train acc 0.883, test acc 0.863, time 3.4 sec
epoch 10, loss 0.0430, train acc 0.886, test acc 0.850, time 3.1 sec


KeyboardInterrupt: 

In [84]:
out = net(test_text,test_keyword)

In [85]:
#np.save('test_out',out.detach().numpy())
#out = np.load('../input/output/output.npy')
out = out.detach().numpy()


In [86]:
out.argmax(axis=1).sum()

1177

In [87]:
submission = pd.DataFrame()
submission['id'] = test_data['id']
submission['target'] = out.argmax(axis=1)

submission.to_csv('submission.csv', index = False)
print('Submission saved')

Submission saved


In [50]:
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
